# Algoritmo TCC

Este notebook foi executado no seguinte ambiente
* Python 3.7.X
* Jypyter Notebook 2.1.2
* pandas 1.0.3
* numpy 1.18.4

Antes de executar verifique os requisitos acima e descomprima o arquivo <code>datasets_consolidados.zip</code>.

In [1]:
import os
import numpy as np
import pandas as pd

Carrega arquivos originais de temperatura e log de servidores.

In [2]:
temp = pd.read_csv('datasets_consolidados/servidores-inlettemp.csv')
logs = pd.read_csv('datasets_consolidados/logs_servidores.csv')

Uma amostra das bases de logs e temperatura de servidores em estado original

In [3]:
temp

,Server,Average,Peak,Time
0,server01,21,22,Fri Dec 20 21:10:46 2013
1,server01,-128,-128,Fri Dec 20 22:10:46 2013
2,server01,-128,-128,Fri Dec 20 23:10:46 2013
3,server01,-128,-128,Sat Dec 21 00:10:46 2013
4,server01,-128,-128,Sat Dec 21 01:10:46 2013
...,...,...,...,...
1541371,server30,19,20,Thu Jan 2 06:01:57 2020
1541372,server30,19,20,Thu Jan 2 07:01:57 2020
1541373,server30,19,21,Thu Jan 2 08:01:57 2020
1541374,server30,18,21,Thu Jan 2 09:01:57 2020


Verifica a quantidade de linhas com valores nulos

In [4]:
temp.isnull().sum()

Server     0
Average    0
Peak       0
Time       0
dtype: int64

In [6]:
logs

,Server,Severity,Date,Description
0,server01,Normal,Fri Dec 20 2013 20:15:46,Log cleared.
1,server01,Critical,Mon Mar 31 2014 14:24:53,The power input for power supply 2 is lost.
2,server01,Critical,Mon Mar 31 2014 14:25:04,Power supply redundancy is lost.
3,server01,Normal,Mon Mar 31 2014 13:29:08,The input power for power supply 2 has been re...
4,server01,Normal,Mon Mar 31 2014 13:29:13,The power supplies are redundant.
...,...,...,...,...
6363,server30,Critical,Tue May 21 2019 13:35:57,Power supply redundancy is lost.
6364,server30,Normal,Tue May 21 2019 20:05:01,The input power for power supply 2 has been re...
6365,server30,Normal,Tue May 21 2019 20:05:07,The power supplies are redundant.
6366,server30,Critical,Sun Jul 28 2019 06:42:53,The power input for power supply 1 is lost.


In [7]:
logs.isnull().sum()

Server         0
Severity       0
Date           0
Description    0
dtype: int64

Remove as não-ocorrências de temperatura, ou seja, o servidor está desligado e isso é evidenciado com o atributo <code>Average</code> com valor de <code>-128</code>.

In [9]:
temp.drop(temp.index[temp['Average'] == -128], inplace = True)

Transforma o atributo <code>Time</code> em tipo Date and cria um atributo <code>DateOnly</code> para posteriores comparações entre bases.

In [10]:
temp['Time']= pd.to_datetime(temp['Time'], format="%a %b %d %H:%M:%S %Y")
temp['DateOnly'] = temp['Time'].dt.date

Ordena a base de temperaturas em função do nome do servidor e do dia/hora.

In [ ]:
temp.sort_values(by=['Server', 'Time'])

Agrupa as linhas em função do nome de servidor e data, sumariza atráves do agrupamento os atributos <code>Average</code> e <code>Peak</code> com informações de Minímo, Máximo, Média, Mediana, Variância e Desvio Padrão.

In [ ]:
temp_summary = temp.groupby(['Server','DateOnly'])[['Average','Peak']].agg(['min', 'max','mean','median','var','std']).reset_index()

A operação de groupby adiciona uma nova linha de indíces, pra isso é necessário realizar um ajuste para se manter uma só linha de índice sem perda de identidade das colunas.

In [ ]:
temp_summary.columns = ["_".join(x) for x in temp_summary.columns.ravel()]
temp_summary.rename(columns={'Server_':'Server','DateOnly_':'DateOnly'},inplace=True)

Remove as ocorrências da base de logs aonde a data tem formato inválido pois indica o System Boot da máquina e transforma o atributo <code>Date</code> em tipo Date.

In [ ]:
logs.drop(logs.index[logs['Date'] == 'System Boot'], inplace = True)
logs['Date']= pd.to_datetime(logs['Date'], format="%a %b %d %Y %H:%M:%S")

Cria o atributo <code>DateOnly</code> e orderna a base de logs em função do nome do servidor e o dia.

In [ ]:
logs['DateOnly'] = logs['Date'].dt.date
logs.sort_values(by=['Server', 'Date'])

Combina as duas bases em uma só usando como índice o nome do servidor e o dia. Para cada ocorrência de log será concatenado os atributos de temperatura do servidor no dia.

In [ ]:
merged2 = pd.merge(temp_summary,logs, on=['Server', 'DateOnly'], how='left')

Substitui o valores vazios (ausência de registro de log) da coluna <code>Severity</code> com o valor <code>Normal</code>

In [ ]:
merged2.Severity.fillna("NoneAlert", inplace=True)

Adiciona o atributo categórico <code>TempSala</code> que indica se a sala de servidores estava quente, esquentando ou fria.

In [ ]:
merged2.loc[merged2['Peak_max'] <= 26, 'TempSala'] = 'SalaFria'
merged2.loc[(merged2['Peak_max'] > 26) & (merged2['Peak_max'] <= 33), 'TempSala'] = 'SalaEsquentando'
merged2.loc[merged2['Peak_max'] > 33, 'TempSala'] = 'SalaQuente'

Amostra da base combinada

In [ ]:
merged2

O atributo Severity será útil ao executar alguns modelos como o PCA na base merged, mas a maioria dos modelos requer que atributos categoricos sejam transformados em inteiros. Para isso, utilizaremos a técnica de One-Hot Encoding para isso.

Abaixo podemos visualizar os possíveis valores para Severity.


In [ ]:
merged2.Severity.unique()

Cria-se um dataset com os valores possíveis e utilizamos a função pd.get_dummies() para gerar as colunas numéricas.

In [ ]:
df = pd.DataFrame({'Severity': ['NoneAlert','Normal', 'Critical', 'Warning']})
merged2 = pd.concat([merged2,pd.get_dummies(merged2['Severity'], prefix='Severity')],axis=1)

Uma amostra dos novos atributos.

In [ ]:
merged2.filter(regex=("Severity_*"))

Exporta as bases para CSV

In [ ]:
#temp.to_csv(r'datasets_consolidados/temperatura.csv', index = False) # export all datasets
#logs.to_csv(r'datasets_consolidados/logs.csv', index = False)
#merged.to_csv(r'datasets_consolidados/merged.csv', index = False)
merged2.to_csv(r'datasets_consolidados/merged2.csv', index = False)

## Avaliando dados com PCA

Frequência de atributos importantes

In [ ]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
Severity = sns.countplot(merged2['Severity'])

In [ ]:
TempSala = sns.countplot(merged2['TempSala'])

Mostra a matriz de correlação entre os atributos núméricos. Cores quentes (vermelho) indicam uma correlação positiva, cores frias (azul) indicam uma correlação negativa.


In [ ]:
import matplotlib.pyplot as plt

corrs = merged2.corr()
mask = np.zeros_like(corrs)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(corrs, cmap='Spectral_r', mask=mask, square=True, vmin=-.4, vmax=.4)
plt.title('Matriz de Correlação')



Usando a temperatura Média (min/max) e Picos (min/max) como features e normalizando os valores

In [ ]:
features = ['Average_max','Average_mean','Peak_max']
# Separating out the features
x = merged2.loc[:, features].values
# Separating out the target
y = merged2.loc[:,['Severity']].values

x = StandardScaler().fit_transform(x)
pd.DataFrame(data = x, columns = features).head()

Executar o PCA

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
principalDf.head(5)

Dataset com os componentes e o target

In [ ]:
finalDf = pd.concat([principalDf, merged2[['Severity']]], axis = 1)
finalDf.head(5)

Plotando o gráfico

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 Component PCA', fontsize = 20)


targets = [ 'NoneAlert','Normal', 'Warning','Critical']
colors = [ 'b','g', 'y','r']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['Severity'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

Criando base de treino e teste 70% treino e 30% teste

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

Normaliza os atributos

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Roda o classificador de árvore de decisão.

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
y_pred

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Matrix de confusão

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)